## Домашнее творческое задание по дисциплине "Технологии обработки больших данных" (часть 2)

Выполнил: студент группы __ПМ20-4 Беленький Михаил Александрович__

Вариант: __10__

    

In [1]:
import pandas as pd
import numpy as np

import dask
import dask.array as da
import dask.bag as db
import dask.dataframe as dd

import re

import sqlite3

import multiprocessing

import h5py

import json

import ast

In [2]:
%load_ext line_profiler

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy`, `pandas` и `dask`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy`, структур `pandas` или структур `dask` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

__В задачах, связанных с пакетом `dask`, явно указано, какой коллекцией вы должны пользоваться при решении задачи. Все операции вычислений должны проводиться  над этой коллекцией и средствами пакета `dask`, если в задании не сказано обратного.__ 



__Задание 1__. В файле `scores.npy` содержится массив оценок рейтингов вопросов. Напишите функцию, которая генерирует строку заданной длины на основе набора символов, считает, сколько раз буквы "п", "и", "т", "о" и "н" суммарно встретились в этой строке, и возвращает это число вместе с полученной строкой. Продемонстрируйте работу этой функции. Сгенерируйте строку длиной 200 тыс. символом на основе символов русского алфавита. Измерьте скорость выполнения этой функции. Получите отчет о построчном выполнении этой функции при помощи `line_profiler` и сохраните его в виде текстовой ячейки.

In [3]:
scores = np.load('C:/Users/micha/Downloads/data/scores.npy')

In [3]:
rus_letters = [chr(letter) for letter in range(1072, 1104)]

In [4]:
def generate_string(length):
    string = ''.join(np.random.choice(rus_letters, length))
    python_letters = len(re.findall(r'[питон]', string))
    return python_letters, string

In [5]:
generate_string(300)

(52,
 'вэрщнйчиъчеьжшжябччужыбвфвивролицхыжфяюшксцъйчьяфякюйаягспвнттдндйзнипцэрдгмсиайарщщььетоютоимифулпывцусщсивчшыорийдабрюямъоыогфиюцееэдооцоеасыьнчсйуцдитзвкбтоэйясшчрлтщэщлзбпшчзсурццяцххыбьыэкьжлцсяпьеюящиюезхцхшвечяыгмалиьящхщаювнъжйэгйндсоыкжакнрюгжрщуйъжсннчфввашиьдфкуэршхжыуиюоэщгиепфлаыааккюхо')

In [6]:
%lprun -f generate_string generate_string(200_000);

```
Timer unit: 1e-07 s

Total time: 0.0639296 s
File: C:\Users\micha\AppData\Local\Temp/ipykernel_45072/635105578.py
Function: generate_string at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def generate_string(length):
     2         1        103.0    103.0      0.0      letters = [chr(letter) for letter in range(1072, 1104)]
     3         1     576647.0 576647.0     90.2      string = ''.join(np.random.choice(letters, length))
     4         1      62537.0  62537.0      9.8      python_letters = len(re.findall(r'[питон]', string))
     5         1          9.0      9.0      0.0      return python_letters, string
```

__Задание 2__. В файле `questions.csv` содержатся вопросы, заданные на сайте stackoverflow. Создайте базу данных SQLite с таблицей `Questions`. Напишите и выполните запрос, который возвращает ID пользователей, у которых есть хоть один вопрос со score, равным 1. Для выполнения запроса используйте функцию `pd.read_sql_query` 

In [7]:
questions_df = pd.read_csv('C:/Users/micha/Downloads/data/questions.csv')

In [8]:
questions_df.head()

,Id,OwnerUserId,CreationDate,Score,Title,Body
0,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...
1,502,147.0,2008-08-02T17:01:58Z,27,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...
2,535,154.0,2008-08-02T18:43:54Z,40,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...
3,594,116.0,2008-08-03T01:15:08Z,25,cx_Oracle: How do I iterate over a result set?,<p>There are several ways to iterate over a re...
4,683,199.0,2008-08-03T13:19:16Z,28,Using 'in' to match an attribute of Python obj...,<p>I don't remember whether I was dreaming or ...


In [143]:
# questions_df['CreationDate'] = pd.to_datetime(questions_df['CreationDate'], format='%Y-%m-%dT%H:%M:%SZ')
# questions_df['OwnerUserId'] = questions_df['OwnerUserId'].astype('Int64')

In [9]:
questions_df.dtypes

Id                int64
OwnerUserId     float64
CreationDate     object
Score             int64
Title            object
Body             object
dtype: object

In [10]:
con = sqlite3.connect('C:/Users/micha/Downloads/data/stackoverflow.db')
cur = con.cursor()

In [11]:
cur.execute("DROP TABLE Questions")
con.commit()

In [12]:
cur.executescript("""
CREATE TABLE IF NOT EXISTS Questions(
    Id INT PRIMARY KEY,
    OwnerUserId VARCHAR,
    CreationDate TEXT,
    Score INT,
    Title TEXT,
    Body TEXT
    );
    """)
con.commit()

In [13]:
query = "INSERT or IGNORE INTO Questions VALUES (?, ?, ?, ?, ?, ?)"
cur.executemany(query, questions_df[['Id', 'OwnerUserId', 'CreationDate', 'Score', 'Title', 'Body']].values)
con.commit()

In [14]:
main_query = """ SELECT OwnerUserId FROM Questions GROUP BY OwnerUserId HAVING Score = 1 """

In [15]:
pd.read_sql_query(sql=main_query, con=con)

,OwnerUserId
0,1000026.0
1,1000161.0
2,1000179.0
3,1000188.0
4,1000389.0
...,...
50132,999657.0
50133,999755.0
50134,999779.0
50135,999808.0


In [16]:
questions_df[questions_df.OwnerUserId == 999755]

,Id,OwnerUserId,CreationDate,Score,Title,Body
172904,16526711,999755.0,2013-05-13T16:19:03Z,1,python mmap regex searching common entries in ...,<p>I have 2 huge xml files. One is around 40GB...
172922,16528178,999755.0,2013-05-13T17:55:17Z,0,python overwrite bytes from offset1 to offset2,"<p>I've seen multiple tutorials in python, pla..."
180105,17074492,999755.0,2013-06-12T20:14:49Z,0,python 2D matrix simple logic works but matrix...,<p>Must be something really stupid that I am d...


__Задание 3__. В файле `questions.csv` содержатся вопросы, заданные на сайте stackoverflow. Используя `multiprocessing.Pool` выясните, сколько вопросов задали в 2011 году. При решении задачи разделите файл на части, для обработки каждой из частей используйте параллельные вычисления.

In [17]:
for index, chunk in enumerate(pd.read_csv('C:/Users/micha/Downloads/data/questions.csv', chunksize=60_000)):
    chunk.to_csv(f'C:/Users/micha/Downloads/data/questions_{index}.csv', index=None)

In [18]:
files = ['questions_0.csv', 'questions_1.csv', 'questions_2.csv', 'questions_3.csv', 'questions_4.csv', 'questions_5.csv', 'questions_6.csv', 'questions_7.csv', 'questions_8.csv', 'questions_9.csv', 'questions_10.csv']

In [19]:
%%file count_2011.py
import pandas as pd
def count_2011(file: str):
    df = pd.read_csv(f'C:/Users/micha/Downloads/data/{file}')
    df['CreationYear'] = pd.to_datetime(df['CreationDate'], format='%Y-%m-%dT%H:%M:%SZ').dt.year

    return df[df['CreationYear'] == 2011].shape[0]

Overwriting count_2011.py


In [20]:
from count_2011 import count_2011

In [21]:
%%time
with multiprocessing.Pool() as pool:
    res = sum(pool.map(count_2011, files))
res

Wall time: 4.57 s


40910

Для сравнения – код без мультпроцессинга:

In [22]:
%%time
example_df = pd.read_csv(f'C:/Users/micha/Downloads/data/questions.csv')
example_df['CreationYear'] = pd.to_datetime(example_df['CreationDate'], format='%Y-%m-%dT%H:%M:%SZ').dt.year
example_df[example_df['CreationYear'] == 2011].shape[0]

Wall time: 10.8 s


40910

__Задание 4__. В файле `qembeddings.hdf` находятся векторные представления заголовков вопросов, заданные на сайте stackoverflow. В файле `qtitles.pickle` находится список заголовков этих вопросов. Напишите функцию, которая для вопроса с индексом `i` выводит заголовки 5 ближайших вопросов в смысле расстояния между эмбеддингами этих вопросов. Выберите метрику, наиболее подходящую (на ваш взгляд) для решения этой задачи. Для работы с эмбеддингами используйте `dask.array`.

In [23]:
qtitles = pd.read_pickle(f'C:/Users/micha/Downloads/data/qtitles.pickle')

In [24]:
file = h5py.File(f'C:/Users/micha/Downloads/data/qembeddinbgs.h5', 'r')

In [25]:
qembeddings_da = da.from_array(file['embeddings'])
qembeddings_da

dask.array<array, shape=(600000, 312), dtype=float32, chunksize=(100000, 312), chunktype=numpy.ndarray>

$${\displaystyle 1. {\text{ Евклидово расстояние}}=d(p,q)=\sqrt{(p_1-q_1)^2+(p_2-q_2)^2+\dots+(p_n-q_n)^2} = \sqrt{\sum_{k=1}^n (p_k-q_k)^2}}$$

In [26]:
def nearest_question_euclid(i:int, count:int):
    euclid_sums = np.sqrt((qembeddings_da[i]-qembeddings_da)**2).sum(axis=1)
    euclid_n_smallest = euclid_sums.argtopk(-(count+1))[1:].compute()
    return qtitles[i], list(map(lambda x: qtitles[x], euclid_n_smallest))

In [27]:
%%time
nearest_question_euclid(2, 5)

Wall time: 916 ms


('Hue supervisor startup error hadoop working fine - Python packages in hue not working',
 ['Installing Python packages in command prompt not working well',
  'List comprehensions work fine in Python shell but are ignored when running the script',
  'setuptools installed package working in ipython but not from scripts',
  "Python setup.py script isn't installing modules correctly",
  "Pydub of Python doesn't work-Probably ffmpeg installed wrongfully"])

$${\displaystyle 2.{\text{ Метрика }L_1 \text{(манхэттенское расстояние)}}=d_1(\textbf{x},\textbf{y})=\sum_{k=1}^{n}{|x_k - y_k|}, \textbf{x}, \textbf{y} \in \mathbb{R}^n}$$

In [28]:
def nearest_question_l1(i:int, count:int):
    l1_sums = da.absolute(qembeddings_da[i]-qembeddings_da).sum(axis=1)
    l1_n_smallest = l1_sums.argtopk(-(count+1))[1:].compute()
    return qtitles[i], list(map(lambda x: qtitles[x], l1_n_smallest))

In [29]:
%%time
nearest_question_l1(2, 5)

Wall time: 483 ms


('Hue supervisor startup error hadoop working fine - Python packages in hue not working',
 ['Installing Python packages in command prompt not working well',
  'List comprehensions work fine in Python shell but are ignored when running the script',
  'setuptools installed package working in ipython but not from scripts',
  "Python setup.py script isn't installing modules correctly",
  "Pydub of Python doesn't work-Probably ffmpeg installed wrongfully"])

Видим, что в топ-5 наименьших расстояний (в случае Евклидовой метрики и в случае Метрики L1) найдено 4/5 совпадений

__Задание 5__. В файлах `questions_with_answers_*.jsonl` содержатся вопросы, заданные на сайте stackoverflow, и ответы на них. Разбейте вопросы на 3 группы: вопросы, на которые дали меньше 5 ответов; вопросы, на которые дали больше 5 ответов, но меньше 15 ответов; и вопросы, на которые дали 15 ответов или более. Посчитайте количество вопросов в каждой из групп. Для обработки данных используйте `dask bag`.

In [30]:
def json_load(line):    
    line = str(line).replace('\n','')
    try:
        ast.literal_eval(str(line))   # пробуем преобразовать строку в словарь
    except (ValueError, SyntaxError):
        return {'CountOfAnswers': 0} # считаем, что дано 0 ответов – правильнее None, но далее проблемы с расчётами
    else:
        line = ast.literal_eval(str(line))
    line['CountOfAnswers'] = len(line['answers'])
    line.pop('answers')
    line.pop('Title')
    line.pop('Body')
    return (json.loads(re.sub("\'", '"', str(line))))

In [31]:
qwa = db.read_text('C:/Users/micha/Downloads/data/questions_with_answers/questions_with_answers_*.jsonl').map(json_load)

In [32]:
lo_discussed = qwa.filter(lambda line: (line['CountOfAnswers']<5))
mi_discussed = qwa.filter(lambda line: ((line['CountOfAnswers']>=5)&(line['CountOfAnswers']<15)))
hi_discussed = qwa.filter(lambda line: (line['CountOfAnswers']>= 15))

In [33]:
%%time
low, mid, high = dask.compute(lo_discussed.count(), mi_discussed.count(), hi_discussed.count())

Wall time: 55 s


In [34]:
pd.DataFrame.from_dict(dict(zip(['Меньше 5 ответов', 'От 5 до 15 ответов', 'Больше 15 ответов'], [low, mid, high])),
                       orient='index', columns=['Количество вопросов'])

,Количество вопросов
Меньше 5 ответов,386162
От 5 до 15 ответов,13650
Больше 15 ответов,195


__Задание 6__. В файле `questions.csv` содержатся вопросы, заданные на сайте stackoverflow. Посчитайте и выведите на экран, сколько вопросов оставил каждый из пользователей в 2008 году. Для обработки данных используйте `dask.dataframe`.

In [35]:
questions_ddf = dd.read_csv('C:/Users/micha/Downloads/data/questions.csv')

In [36]:
questions_ddf

,Id,OwnerUserId,CreationDate,Score,Title,Body
npartitions=15,,,,,,
,int64,float64,object,int64,object,object
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [37]:
questions_ddf['CreationYear'] = dd.to_datetime(questions_ddf['CreationDate'], format='%Y-%m-%dT%H:%M:%SZ').dt.year

In [38]:
que_2008 = questions_ddf[questions_ddf['CreationYear']==2008].groupby('OwnerUserId').count()

In [39]:
%%time
que_2008.compute()

Wall time: 9.13 s


,Id,CreationDate,Score,Title,Body,CreationYear
OwnerUserId,,,,,,
30.0,1,1,1,1,1,1
38.0,1,1,1,1,1,1
59.0,1,1,1,1,1,1
61.0,1,1,1,1,1,1
64.0,1,1,1,1,1,1
...,...,...,...,...,...,...
2007483.0,2,2,2,2,2,2
2025531.0,1,1,1,1,1,1
2220518.0,12,12,12,12,12,12
